<a href="https://colab.research.google.com/github/Hab-eeb/Flower_color_detection/blob/main/color_det_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import os
import sys
import datetime
import glob as glob
import numpy as np
import cv2
from collections import Counter
import shutil

# import the Keras implementations of VGG16, VGG19, InceptionV3 and Xception models
import tensorflow 
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications import InceptionV3, ResNet50, Xception, vgg16
from tensorflow.keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
from keras import layers, models, optimizers
import tensorflow.keras.backend as K
import itertools
from itertools import tee
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

#from scipy.interpolate import spline
import pandas as pd
import matplotlib.pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.utils import class_weight

# Importing Dataset from google drive

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! unzip "/content/drive/MyDrive/Flower_Color_Detection/Orchid Flowers Dataset-v1.1.zip"

# Model Building

In [3]:
# Definition for calculating class weights
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight( class_weight='balanced',classes = [0.,1.], y = y_true[:, i])
    return weights

In [4]:
# weighted binary cross entropy definition
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

In [5]:
 
images_dir = "/content/Orchid Flowers Dataset-v1.1/Orchid_Images"
# Using CF1 PRIMARY version 2 color classifier

# Create training images folder 
ndf = pd.read_csv("/content/Orchid Flowers Dataset-v1.1/Color_Classifier/Version 2/Primary_Color/CF1/Training_Data_for_Color_of_Flower.txt",header=None)
if os.path.exists("/content/train"):
  shutil.rmtree("/content/train")
for ind in ndf.index:
  if os.path.exists(os.path.join("/content/train",str(ndf[1][ind]))):
    shutil.copy2(os.path.join(images_dir,str(ndf[0][ind])),os.path.join("/content/train",str(ndf[1][ind])))
  else :
    os.makedirs(os.path.join("/content/train",str(ndf[1][ind])))
    shutil.copy2(os.path.join(images_dir,str(ndf[0][ind])),os.path.join("/content/train",str(ndf[1][ind]))) 

# Create validation images folder
ndf = pd.read_csv("/content/Orchid Flowers Dataset-v1.1/Color_Classifier/Version 2/Primary_Color/CF1/Validation_Data_for_Color_of_Flower.txt",header=None)
if os.path.exists("/content/validate"):
  shutil.rmtree("/content/validate")
for ind in ndf.index:
  if os.path.exists(os.path.join("/content/validate",str(ndf[1][ind]))):
    shutil.copy2(os.path.join(images_dir,str(ndf[0][ind])),os.path.join("/content/validate",str(ndf[1][ind])))
  else :
    os.makedirs(os.path.join("/content/validate",str(ndf[1][ind])))
    shutil.copy2(os.path.join(images_dir,str(ndf[0][ind])),os.path.join("/content/validate",str(ndf[1][ind]))) 

# Create Testing images folder
ndf = pd.read_csv("/content/Orchid Flowers Dataset-v1.1/Color_Classifier/Version 2/Primary_Color/CF1/Testing_Data_for_Color_of_Flower.txt",header=None)
if os.path.exists("/content/test"):
  shutil.rmtree("/content/test")
for ind in ndf.index:
  if os.path.exists(os.path.join("/content/test",str(ndf[1][ind]))):
    shutil.copy2(os.path.join(images_dir,str(ndf[0][ind])),os.path.join("/content/test",str(ndf[1][ind])))
  else :
    os.makedirs(os.path.join("/content/test",str(ndf[1][ind])))
    shutil.copy2(os.path.join(images_dir,str(ndf[0][ind])),os.path.join("/content/test",str(ndf[1][ind]))) 



In [6]:
# [Dataset]
# image dimensions for VGG16, VGG19 are 224, 224
# image dimensions for InceptionV3 and Xception are 299, 299
img_width, img_height = 224,224

train_dir = '/content/train'
validate_dir = '/content/validate'
nb_epochs = 10
batch_size = 64
nb_classes = len(glob.glob(train_dir + '/*'))

# get number of images in training directory
nb_train_samples = 0
for r, dirs, files in os.walk(train_dir):
    for dr in dirs:
        nb_train_samples += len(glob.glob(os.path.join(r, dr + "/*")))

# get number of images in validation directory
nb_validate_samples = 0
for r, dirs, files in os.walk(validate_dir):
    for dr in dirs:
        nb_validate_samples += len(glob.glob(os.path.join(r, dr + "/*")))

print(nb_train_samples)
print(nb_validate_samples)
print(nb_classes)

5119
1235
4


In [7]:
# data pre-processing for training
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

# data pre-processing for validation
validate_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

testdatagen=ImageDataGenerator(rescale=1./255)

train_gener =train_datagen.flow_from_directory(
train_dir,
target_size=(img_height,img_width),
#the total amount should be divisable by batch size
batch_size=5119,
shuffle=False,
class_mode= "categorical" )

Found 5119 images belonging to 4 classes.


In [8]:
# generate and store training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       

(train_pics,train_labels) = next(train_gener)
class_weights = calculating_class_weights(train_labels) 

# generate and store validation data
validate_generator = validate_datagen.flow_from_directory(
    validate_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

Found 5119 images belonging to 4 classes.
Found 1235 images belonging to 4 classes.


In [9]:
#%% define the model (see other evaluation_load file for the other models, and you can copy paste them here to use them)
vgg16model = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(img_width, img_height,3)
)
for layer in vgg16model.layers[:2]:
    layer.trainable=False
    
vgg16model.summary()

x= vgg16model.output
x = Flatten(name='flatten')(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
predictions= Dense(nb_classes,activation='softmax')(x)

model= Model(inputs=[vgg16model.input], outputs= predictions)

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
# To make sure you can see the f1, precision and recall scores of the validation set during training.
class Metrics(Callback):
    def __init__(self, validation_generator, validation_steps, threshold=0.5):
        self.validation_generator = validation_generator
        self.validation_steps = validation_steps or len(validation_generator)
        self.threshold = threshold

    def on_train_begin(self, logs={}):
        self.val_f1_scores = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        # duplicate generator to make sure y_true and y_pred are calculated from the same observations
        gen_1, gen_2 = tee(self.validation_generator)
        y_true = np.vstack(next(gen_1)[1] for _ in range(self.validation_steps)).astype('int')
        y_pred = (self.model.predict_generator(gen_2, steps=self.validation_steps) > self.threshold).astype('int')
        f1 = f1_score(y_true, y_pred, average='macro')
        precision = precision_score(y_true, y_pred, average='macro')
        recall = recall_score(y_true, y_pred, average='macro')
        self.val_f1_scores.append(f1)
        self.val_recalls.append(recall)
        self.val_precisions.append(precision)
        print(f" - val_f1_score: {f1:.5f} - val_precision: {precision:.5f} - val_recall: {recall:.5f}")
        return 

checkpoint = ModelCheckpoint( "weights3.{epoch:02d}.hdf5", verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)

In [11]:
## Training


model.compile(loss=get_weighted_loss(class_weights), optimizer=tensorflow.keras.optimizers.Adam(learning_rate = 0.00005), metrics = ['accuracy'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validate_generator.n//validate_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

metrics = Metrics(validate_generator, STEP_SIZE_VALID)

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validate_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=nb_epochs,
                # callbacks=[metrics]
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  app.launch_new_instance()


Epoch 1/10
79/79 [==============================] - 146s 2s/step - loss: 0.6665 - accuracy: 0.3234 - val_loss: 0.5645 - val_accuracy: 0.4054
Epoch 2/10
79/79 [==============================] - 111s 1s/step - loss: 0.5608 - accuracy: 0.4645 - val_loss: 0.5074 - val_accuracy: 0.5436
Epoch 3/10
79/79 [==============================] - 112s 1s/step - loss: 0.4962 - accuracy: 0.5735 - val_loss: 0.4260 - val_accuracy: 0.6505
Epoch 4/10
79/79 [==============================] - 111s 1s/step - loss: 0.4298 - accuracy: 0.6671 - val_loss: 0.3675 - val_accuracy: 0.7163
Epoch 5/10
79/79 [==============================] - 109s 1s/step - loss: 0.3815 - accuracy: 0.7225 - val_loss: 0.3705 - val_accuracy: 0.7352
Epoch 6/10
79/79 [==============================] - 110s 1s/step - loss: 0.3303 - accuracy: 0.7569 - val_loss: 0.3254 - val_accuracy: 0.7574
Epoch 7/10
79/79 [==============================] - 111s 1s/step - loss: 0.2950 - accuracy: 0.7810 - val_loss: 0.3286 - val_accuracy: 0.7599
Epoch 8/10
79

In [22]:
#Using a random picture gotten from the internet to predict # 1 is greenish , 2 is pink/purple, 3 is yellowish white, 4 is redish, brown, orange
im2 = cv2.imread("/content/istockphoto-1143546879-612x612.jpg")
im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
im2 = cv2.resize(im2, (img_width, img_height)).astype(np.float32)
im2 = np.expand_dims(im2, axis = 0)/255 

np.argmax(model.predict(im2)) 

2